In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot
import warnings
warnings.filterwarnings("ignore")
from math import radians, cos, sin, asin, sqrt
from sklearn import linear_model
from datetime import timedelta
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [13]:
import os
os.getcwd()

'C:\\Users\\Sayeed Alvi\\Desktop\\LABike_DataScienceCompetition'

In [15]:
#Reading data file 
data = pd.read_csv('df_mod.csv')
data.head()

,trip_id,bike_id,start_station,end_station,trip_route_category,start_time,end_time,start_lat,start_lon,end_lat,end_lon,plan_duration,passholder_type
0,59256078,6417,4159,3042,One Way,21/11/2017 11:42:00 AM,21/11/2017 12:37:00 PM,34.145901,-118.11653,34.049301,-118.23880,0,Walk-up
1,60223612,6453,4159,4138,One Way,27/11/2017 8:13:00 AM,27/11/2017 8:41:00 AM,34.145901,-118.11653,34.145691,-118.14823,0,Walk-up
2,58576402,6650,4159,4138,One Way,17/11/2017 4:39:00 PM,17/11/2017 4:57:00 PM,34.145901,-118.11653,34.145691,-118.14823,30,Monthly Pass
3,51293790,12327,4159,4138,One Way,11/10/2017 16:18,11/10/2017 16:33,34.145901,-118.11653,34.145691,-118.14823,30,Monthly Pass
4,50594191,6000,4159,4138,One Way,8/10/2017 11:51,8/10/2017 12:12,34.145901,-118.11653,34.145691,-118.14823,0,Walk-up


In [17]:
#Creating a new column Date from the column "start_time" of the dataframe 
data["Date"] = pd.to_datetime(data["start_time"])

In [18]:
#Creating a copy of data since loading data takes a lot of time
data1 = data.copy()

In [19]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 586807 entries, 0 to 586806
Data columns (total 14 columns):
trip_id                586807 non-null int64
bike_id                586807 non-null object
start_station          586807 non-null int64
end_station            586807 non-null int64
trip_route_category    586807 non-null object
start_time             586807 non-null object
end_time               586807 non-null object
start_lat              586807 non-null float64
start_lon              586807 non-null float64
end_lat                586807 non-null float64
end_lon                586807 non-null float64
plan_duration          586807 non-null int64
passholder_type        586807 non-null object
Date                   586807 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(4), object(5)
memory usage: 62.7+ MB


In [20]:
#Dropping null values
data1 = data1.dropna() 
# Converting float values to interger
# data1["end_station"] = data1["end_station"].astype(int)  
data1 = data1.set_index("Date") #setting index to Date column
data1 = data1.sort_index() #Arranging dates 

In [21]:
data1.head()

,trip_id,bike_id,start_station,end_station,trip_route_category,start_time,end_time,start_lat,start_lon,end_lat,end_lon,plan_duration,passholder_type
Date,,,,,,,,,,,,,
2016-01-08 00:12:00,4767292,6127,3034,3055,One Way,1/8/2016 0:12,1/8/2016 0:49,34.042061,-118.26338,34.044159,-118.25158,0,Walk-up
2016-01-08 00:13:00,4767291,6026,3034,3055,One Way,1/8/2016 0:13,1/8/2016 0:49,34.042061,-118.26338,34.044159,-118.25158,0,Walk-up
2016-01-08 00:18:00,4767290,5990,3021,3022,One Way,1/8/2016 0:18,1/8/2016 0:43,34.045609,-118.23703,34.046070,-118.23309,30,Monthly Pass
2016-01-08 00:23:00,4767289,6585,3063,3064,One Way,1/8/2016 0:23,1/8/2016 1:46,34.049198,-118.25283,34.046810,-118.25698,0,Walk-up
2016-01-08 00:25:00,4767288,6069,3063,3064,One Way,1/8/2016 0:25,1/8/2016 1:46,34.049198,-118.25283,34.046810,-118.25698,0,Walk-up


In [22]:
#length of end station is 142 and length of start station is 140

"""As length is not same of these two list, a search method is used to identify which element 
is missing from the start station, hence the next chunk of code is to find
the station number not in the start station list"""
remove_stn = []
for i in range(len(end_stn)):
    if end_stn[i] not in start_stn:
        remove_stn.append(end_stn[i])
#Updating end station list
end_stn = [stn for stn in end_stn if stn not in remove_stn]

NameError: name 'end_stn' is not defined

In [25]:
#Removing rows with station number 4110,4143 and 4118 as it is discarded from the analysis
data1 = data1[data1['end_station']!=4110]
data1 = data1[data1['end_station']!=4118]
data1 = data1[data1['start_station']!=4143]
data1.shape

(586556, 13)

In [26]:
#Setting plan_duration = 0 for passholder_type = Walk-up
data1['plan_duration'].loc[data1['passholder_type'] == "Walk-up"] = 0

#Setting annual pass = flex as both of them are the same
data1['passholder_type'].loc[data1['passholder_type'] == "Annual Pass"] = "Flex Pass"

In [ ]:
#Calculating trip duration in minutes
data1['start_time']= pd.to_datetime(data1['start_time']) 
data1['end_time']= pd.to_datetime(data1['end_time']) 
data1['trip_duration_mins'] = (data1.end_time - data1.start_time)/ timedelta(minutes=1)

In [421]:
#Creating a copy of data since it takes time to load data
data2 = data1.copy()

In [422]:
#Create dummy variables for passholder type
one_hot_pass = pd.get_dummies(data2['passholder_type']).rename(columns={
            'Flex Pass': 'annual',
            'Monthly Pass': 'monthly',
            'One Day Pass': 'one_day',
            'Walk-up':'walk_up'})

#Create dummy variables for trip route category
one_hot_trip_type = pd.get_dummies(data2['trip_route_category']).rename(columns={
            'Round Trip': 'round_trip',
            'One Way': 'one_way'})

In [423]:
#Calculating Distance traveled 
def calc_dist(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [424]:
vfunc = np.vectorize(calc_dist)
data2['Distance']= vfunc(data2.start_lon,data2.start_lat,data2.end_lon,data2.end_lat)

In [426]:
month = 'month'
week = 'week'
day = 'day'
hour = 'hour'
def grouping(data,col):
    '''
    Stacking dataset on the basis of start_station
    Args: data = dataframe
          col = column name/ time frame
    Returns: Stacked dataframe with index resetted
    '''
    df = data.copy()
    df["Year"]=df.start_time.dt.year
    if col.lower() == 'month':
        col = col.capitalize()
        df[col] = df.start_time.dt.month
    elif col.lower() == 'week':
        col = col.capitalize()
        df[col] = df.start_time.dt.week
    elif col.lower() == 'day':
        col = col.capitalize()
        df[col] = df.start_time.dt.day
    elif col.lower() == 'hour':
        col = col.capitalize()
        df[col] = df.start_time.dt.hour
    else:
        return print('Unable to extract time from index!!')
    df_sum = df.groupby(by=["start_station","Year",col]).sum()[['Distance', 'trip_duration_mins']]
    df_count = df.groupby(by=["start_station","Year",col]).count()[['trip_id']]
    df = pd.concat([df_sum, df_count], axis=1)
    df = df.reset_index()
    return df

In [428]:
data3 = grouping(data2,month)

In [429]:
#Setting offset for continuity in months
data3.loc[data3['Year']==2017, 'Month'] = data3['Month']+12
data3.loc[data3['Year']==2018, 'Month'] = data3['Month']+24

In [448]:
station = list(set(data3.start_station))
r_squared = []
#Looping over all the stations to fit linear regressions individually
for i in range(len(station)):
    stn = station[i]
    df = data3.loc[data3['start_station']==stn]
    x = df[['Distance','trip_duration_mins']]
    y = df["trip_id"]
    X_train, X_test, y_train, y_test = train_test_split( #Splitting data into train & test
            x, y, test_size=0.2, random_state=42)
    reg = linear_model.LinearRegression()
    est2 = reg.fit(x,y)
    ypred = est2.predict(X_test) #predicting y values on test dataset
    score = r2_score(y_test,ypred) #Calculating r-squared value on the test data
    r_squared.append(score)
r_squared

[0.977042164894722,
 0.9681489740558211,
 0.9951209706972303,
 0.8771429239771356,
 0.9714272789367847,
 -3.3907828833157723,
 0.8442937381930394,
 -0.8622901242799996,
 0.9709507336078433,
 -0.11038563075151453,
 -0.35133007659479354,
 0.5496902263221386,
 0.6988038961213752,
 0.7922317474378195,
 0.5610530423416575,
 0.6775364037357341,
 0.6754062268673511,
 0.7084018211068092,
 0.5951131395256846,
 0.6215908032943063,
 0.7914509174496147,
 0.9781397137736292,
 0.8733219167217421,
 0.9230401287197263,
 0.8786493615933924,
 0.9709783528694647,
 0.9045155868153902,
 0.9830458701303861,
 0.9846006003425768,
 0.9906257184785754,
 0.8940165542515042,
 0.6592646499398068,
 0.935610348223027,
 0.9869397785216414,
 0.9964337195418876,
 0.9425070499448821,
 0.9224509512598666,
 0.9506369925250449,
 0.9018091641159793,
 0.8831470129994927,
 0.9922805474785648,
 0.9719919812336326,
 0.9692697232185683,
 0.7146358031568936,
 0.8123610383458099,
 0.9794548815408723,
 0.9478830821974717,
 0.914626